In [ ]:
import os
os.chdir('..')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import numpy as np
import pandas as pd
import yaml
import os
%load_ext autoreload
%autoreload 2

In [ ]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=100, num_workers=6)

In [ ]:
videos, answers, hidden_states, vid_folder  = iter(dataloader).next()

In [ ]:
MODEL_VERSION = 5
hparams_path = f'./lightning_logs/version_{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/')[0]
ckpt_file_path = f'./lightning_logs/version_{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
model = LitModule.load_from_checkpoint(ckpt_file_path)

In [ ]:
model_out = model(videos)
df_hidden_states_pred = pd.DataFrame(model_out.detach().numpy(), columns=const.HIDDEN_STATE_COLS)

In [ ]:
df_hidden_states = pd.DataFrame(hidden_states.detach().numpy(), columns=const.HIDDEN_STATE_COLS)

In [ ]:
#alt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=4, cols=1)

for i, col in enumerate(const.HIDDEN_STATE_COLS):
        fig.add_trace(go.Scatter(x=df_hidden_states[col], y=df_hidden_states_pred[col],
                            mode='markers', 
                                marker_color='#1f77b4'),
                     row=i+1, col=1)

fig.update_layout(height=1000, width=1200, title_text="Latent neuron activations vs. hidden states", showlegend=False)
fig.show()